In [1]:
import scanpy as sc
import numpy as np
import csv
import pickle
import matplotlib
import math
from itertools import combinations
import pandas as pd
import matplotlib
# from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
# from cdlib import algorithms
# from cdlib import NodeClustering


spot_diameter = 89.43 #pixels

In [2]:
def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df

In [3]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)

# Load connected_components (assume already have it)
with open("/Users/victoriagao/local_docs/NEST/stored_variables/filtered_connected_components.pkl", 'rb') as file:
    filtered_connected_components = pickle.load(file)

# Load pathologist's label
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/input/pathologist_annot/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

In [4]:
filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
inputFile = '/Users/victoriagao/local_docs/NEST/output/From_Fatema/'+filename_str
df = pd.read_csv(inputFile, sep=",")
csv_record_final = df.values.tolist()
df_column_names = list(df.columns)
csv_record_final = [df_column_names] + csv_record_final

In [10]:
#### i.e. making a list of Dictionary to store matched records for each region
matched_records_set_lib = {}  

for set_index, connected_comp_set in enumerate(filtered_connected_components):
    matched_records = []

    for record in csv_record_final:
        from_cell, to_cell = record[0], record[1]
        # if (from_cell in connected_comp_set) and (to_cell in connected_comp_set): # this is for tumor v tumor spots interaction
        #     matched_records.append(record)
        if (from_cell in connected_comp_set) ^ (to_cell in connected_comp_set): # this is for tumor v stroma/stroma v tumor spots interaction, XOR argument
            matched_records.append(record)
    
    # Store matched records for this set index in the dictionary
    matched_records_set_lib[set_index+1] = matched_records

In [11]:
# Calculate Hill number q=1 for all 21 regions
columns = ['from_cell','to_cell','ligand','receptor','attention_score','component','from_id','to_id']

LR_freq_data = []

for key, value in matched_records_set_lib.items():
    df_matched = pd.DataFrame(value, columns=columns)
    df_matched_processed = preprocessDf(df_matched)
    ligand_receptor_counts = df_matched_processed['ligand-receptor'].value_counts()
    count_df = pd.DataFrame({'ligand-receptor': ligand_receptor_counts.index, 'count': ligand_receptor_counts.values})
    total_counts = count_df['count'].sum()
    print("For set: "+str(key))
    print("LR frequency",total_counts)

    LR_freq_data.append({'Set': key, 'LR_frequency': total_counts}) # putting the hill numbers into a list
    
# Create a DataFrame from the collected data
LR_freq_df = pd.DataFrame(LR_freq_data)
LR_freq_dict = LR_freq_df.set_index('Set')['LR_frequency'].to_dict()

For set: 1
LR frequency 20
For set: 2
LR frequency 14
For set: 3
LR frequency 175
For set: 4
LR frequency 49
For set: 5
LR frequency 64
For set: 6
LR frequency 2
For set: 7
LR frequency 7
For set: 8
LR frequency 20
For set: 9
LR frequency 5
For set: 10
LR frequency 0
For set: 11
LR frequency 10
For set: 12
LR frequency 0
For set: 13
LR frequency 5
For set: 14
LR frequency 5
For set: 15
LR frequency 4
For set: 16
LR frequency 1
For set: 17
LR frequency 1
For set: 18
LR frequency 5
For set: 19
LR frequency 0
For set: 20
LR frequency 4
For set: 21
LR frequency 0


In [9]:
# save the results
with open('/Users/victoriagao/local_docs/NEST/stored_variables/64630_LR_freq_TS_interact.pickle', 'wb') as file:
    pickle.dump(LR_freq_dict, file)